In [1]:
import os
import pandas as pd

In [5]:
dev_file_path='./dev.json'

In [6]:
df=pd.read_json(dev_file_path)

In [7]:
inter_df=pd.DataFrame(df['data'].iloc[0])
intra_df=pd.DataFrame(df['data'].iloc[1])

In [50]:
def reformat_sentences(df):
    choice_1=[]
    choice_2=[]
    choice_3=[]
    choice_1_gold=[]
    choice_2_gold=[]
    choice_3_gold=[]
    for i in range(len(df)):
        sentences=df['sentences'].iloc[i]
        choice_1.append(sentences[0]['sentence'])
        choice_2.append(sentences[1]['sentence'])
        choice_3.append(sentences[2]['sentence'])
        choice_1_gold.append(sentences[0]['gold_label'])
        choice_2_gold.append(sentences[1]['gold_label'])
        choice_3_gold.append(sentences[2]['gold_label'])
    df['choice_1']=choice_1
    df['choice_1_gold']=choice_1_gold
    df['choice_2']=choice_2
    df['choice_2_gold']=choice_2_gold
    df['choice_3']=choice_3
    df['choice_3_gold']=choice_3_gold
    return df

In [51]:
intra_df=reformat_sentences(intra_df)
inter_df=reformat_sentences(inter_df)

In [80]:
# intra_df.head()

In [53]:
def replace_blank(text):
    return text.replace('BLANK','[MASK]')

In [54]:
intra_df['context']=intra_df['context'].apply(lambda x:replace_blank(x))

In [79]:
# intra_df.head()

## TinyLlama Load and Inference

In [6]:
system_instruction="You are a helpful AI assistant. You are given a sentence followed by choices. Output only one word answer either Choice1 or Choice2 or Choice3 based on your understanding. Do not give extra explanation or related information."

In [66]:
def load_tinyllama(model_file_path):
    ## Tinyllama 1.1 b chat prerequisits
    pipeline = pipeline("text-generation", 
                                  model=model_file_path, 
                                  torch_dtype=torch.bfloat16, device_map=device
                                 )
    return pipeline

In [67]:
def tinyllama_inference_model(question):
    messages_tinyllama = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": question}
    ]        
    prompt=pipeline.tokenizer.apply_chat_template(messages_tinyllama, tokenize=False,
                                                              add_generation_prompt=True)
    outputs = pipeline(prompt, max_new_tokens=3, do_sample=True, 
                                 temperature=1, top_k=50, top_p=0.95)
    res = outputs[0]["generated_text"].split('<|assistant|>')[1]
    res_tinyllama = res.replace('\n', '').replace(' ','')
    return res_tinyllama

## Phi3 Load and inference

In [62]:
def load_phi3(model_file_path):
    ## phi-3.5 mini instruct prerequisites
    torch.random.manual_seed(0)
    model_phi = model_file_path
    model_phi3 = AutoModelForCausalLM.from_pretrained(
        model_phi, 
        device_map=device, 
        torch_dtype="auto", 
        trust_remote_code=True, 
    )
    tokenizer = AutoTokenizer.from_pretrained(model_phi)
    pipeline = pipeline(
        "text-generation",
        model=model_phi3,
        tokenizer=tokenizer,
    )

    return pipeline

In [ ]:
def phi3_inference_model(question):
#     phi3_sample_responses = []
#     for i in range(10):
    messages_phi3 = [
    {"role": "system", "content":system_instruction},
    {"role": "user", "content": question},]
    generation_args = {
        "max_new_tokens": 3,
        "return_full_text": False,
        "temperature": 1,
        "do_sample": True,
    }
    output = pipeline(messages_phi3, **generation_args)
    result = output[0]['generated_text']
#     phi3_sample_responses.append(result)
    return result

## Mistral 7b load and inference

In [ ]:
def load_mistral(model_file_path):
    ## Mistal 7b instruct prerequisites
    mistral_model = AutoModelForCausalLM.from_pretrained(model_file_path)
    mistral_tokenizer = AutoTokenizer.from_pretrained(model_file_path)
    return mistral_model, mistral_tokenizer

In [ ]:
def mistral7b_inference_model(question):
    messages_mistral = [{"role": "system", "content":system_instruction},
                        {"role": "user", "content": question}]
    encodeds = mistral_tokenizer.apply_chat_template(messages_mistral, return_tensors="pt")
    model_inputs = encodeds.to(device)
    mistral_model.to(device)
    generated_ids = mistral_model.generate(model_inputs, max_new_tokens=3, do_sample=True, top_k=50, top_p=0.95, temperature=1)
    decoded = mistral_tokenizer.batch_decode(generated_ids)
    result = (decoded[0].split('[/INST]')[1]).replace('</s>', '') 
    return result

## Llama 8b Load and Inference

In [64]:
def load_llama(model_file_path):
    ## llama3.1 8b instruct prerequisites

    model_llama3 = model_file_path
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_llama3,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map=device,
    )
    return pipeline

In [65]:
def llama3_inference_model(question):
    messages_llama3 = [
    {"role": "system", "content": system_instruction},
    {"role": "user", "content": question},]
    outputs = pipeline(
    messages_llama3,
    max_new_tokens=3,
    temperature=1,
    )
    res = outputs[0]["generated_text"][-1]
    res_updated = res['content']
    return res_updated


In [57]:
intra_df.shape

(2106, 11)

In [5]:
def get_output(row, data_sample_type):
    # Create the query string based on the prompt type
    context=row['context']
    choice1=row['choice_1']
    choice2=row['choice_2']
    choice3=row['choice_3']
    if data_sample_type == 'intersentence':
        query = f"""Question: Select the item from this list which relates the most to the context.
        Context: {context}
        Choice1: {choice1}
        Choice2: {choice2}
        Choice3: {choice3}"""
    elif data_sample_type == 'intrasentence':
        query = f"""Question: The context is missing a word defined as [MASK]. Select which sentence
        among the 3 choices below is the best way to replace it.
        Context: {context}
        Choice1: {choice1}
        Choice2: {choice2}
        Choice3: {choice3}"""

    output = model_inference_function(query)

    return output

In [78]:
def generate_output(model_name,model_file_path):
    if model_name!='mistral':
        model_load_string="load_"+model_name
        model_load_function=eval(model_load_string)
        pipeline = model_load_function(model_file_path)
    
        model_inference_string=model_name+"_inference_model"
        model_inference_function=eval(model_inference_string)
    else:
        mistral_model, mistral_tokenizer = load_mistral(model_file_path)
        model_inference_function=eval("mistral7b_inference_model")
         
    data_sample_types=['intersentence','intrasentence']
    
    for data_sample_type in data_sample_types:
        if data_sample_type=='intersentence':
            df=inter_df
        else:
            df=intra_df
        model_outputs=[]
        for idx, row in df.iterrows():
            out=get_output(row, data_sample_type)
            model_outputs.append(out)
        df[model_name+'_output']=model_outputs
        df.to_csv(model_name+'_'+data_sample_type+'_Output.csv',index=False)

In [ ]:
# sample usage
generate_output('tinyllama','/opt')